# Liver Model

## Import some things

In [4]:
from Base_Deeplearning_Code.Data_Generators.Generators import Train_Data_Generator2D, os, np, plot_scroll_Image, dice_coef_3D
from Base_Deeplearning_Code.Data_Generators.Image_Processors import *
from Utils import ModelCheckpoint, model_path_maker
from Callbacks.Visualizing_Model_Utils import TensorBoardImage
import tensorflow as tf
import keras.backend as K

In [5]:
%matplotlib inline

In [6]:
%load_ext tensorboard

In [7]:
data_path = os.path.join('..','Data','Niftii_Arrays')
train_path = [os.path.join(data_path,'Train')]
validation_path = os.path.join(data_path,'Validation')
test_path = os.path.join(data_path,'Test')
model_path = os.path.join('..','Models')
if not os.path.exists(model_path):
    os.makedirs(model_path)

## We now need some image processors...

#### We will ensure that the images are 512 x 512, normalize them with a mean of 78 and std of 29, add random noise, and threshold

In [8]:
image_processors_train = [Ensure_Image_Proportions(512,512),Normalize_Images(mean_val=78,std_val=29),
                          Add_Noise_To_Images(variation=np.round(np.arange(start=0, stop=0.3, step=0.1),2)),
                          Threshold_Images(lower_bound=-3.55,upper_bound=3.55)]
image_processors_test = [Ensure_Image_Proportions(512,512),Normalize_Images(mean_val=78,std_val=29),
                         Threshold_Images(lower_bound=-3.55,upper_bound=3.55)]

In [9]:
train_generator = Train_Data_Generator2D(shuffle=True,data_paths=train_path,batch_size=5,image_processors=image_processors_train)

ABD_LYMPH_016
no start or stop
MED_LYMPH_064
no start or stop
MED_LYMPH_063
no start or stop
MED_LYMPH_024
no start or stop
ABD_LYMPH_019
no start or stop
ABD_LYMPH_016
no start or stop
MED_LYMPH_017
no start or stop
ABD_LYMPH_016
no start or stop
MED_LYMPH_017
no start or stop
ABD_LYMPH_033
no start or stop
ABD_LYMPH_007
no start or stop
ABD_LYMPH_033
no start or stop
ABD_LYMPH_016
no start or stop
MED_LYMPH_064
no start or stop
ABD_LYMPH_007
no start or stop
MED_LYMPH_017
no start or stop
ABD_LYMPH_033
no start or stop
ABD_LYMPH_007
no start or stop
ABD_LYMPH_019
no start or stop
MED_LYMPH_017
no start or stop
MED_LYMPH_067
no start or stop
MED_LYMPH_063
no start or stop
MED_LYMPH_063
no start or stop
MED_LYMPH_030
no start or stop
MED_LYMPH_017
no start or stop
MED_LYMPH_024
no start or stop
MED_LYMPH_073
no start or stop
ABD_LYMPH_019
no start or stop
ABD_LYMPH_016
no start or stop
MED_LYMPH_067
no start or stop
ABD_LYMPH_007
no start or stop
MED_LYMPH_030
no start or stop
ABD_LYMP

ABD_LYMPH_016
no start or stop
MED_LYMPH_030
no start or stop
ABD_LYMPH_016
no start or stop
MED_LYMPH_024
no start or stop
MED_LYMPH_024
no start or stop
MED_LYMPH_064
no start or stop
MED_LYMPH_064
no start or stop
MED_LYMPH_063
no start or stop
ABD_LYMPH_016
no start or stop
ABD_LYMPH_019
no start or stop
MED_LYMPH_030
no start or stop
MED_LYMPH_017
no start or stop
MED_LYMPH_063
no start or stop
MED_LYMPH_063
no start or stop
ABD_LYMPH_033
no start or stop
MED_LYMPH_002
no start or stop
ABD_LYMPH_033
no start or stop
MED_LYMPH_055
no start or stop
ABD_LYMPH_007
no start or stop
ABD_LYMPH_019
no start or stop
ABD_LYMPH_016
no start or stop
MED_LYMPH_002
no start or stop
MED_LYMPH_030
no start or stop
ABD_LYMPH_019
no start or stop
MED_LYMPH_024
no start or stop
ABD_LYMPH_033
no start or stop
ABD_LYMPH_016
no start or stop
ABD_LYMPH_033
no start or stop
MED_LYMPH_063
no start or stop
MED_LYMPH_063
no start or stop
ABD_LYMPH_033
no start or stop
MED_LYMPH_055
no start or stop
MED_LYMP

no start or stop
ABD_LYMPH_033
no start or stop
ABD_LYMPH_016
no start or stop
ABD_LYMPH_007
no start or stop
MED_LYMPH_055
no start or stop
MED_LYMPH_030
no start or stop
MED_LYMPH_064
no start or stop
MED_LYMPH_055
no start or stop
ABD_LYMPH_033
no start or stop
ABD_LYMPH_016
no start or stop
ABD_LYMPH_007
no start or stop
MED_LYMPH_067
no start or stop
MED_LYMPH_024
no start or stop
MED_LYMPH_067
no start or stop
MED_LYMPH_002
no start or stop
ABD_LYMPH_019
no start or stop
MED_LYMPH_030
no start or stop
ABD_LYMPH_069
no start or stop
MED_LYMPH_055
no start or stop
ABD_LYMPH_007
no start or stop
ABD_LYMPH_007
no start or stop
MED_LYMPH_017
no start or stop
MED_LYMPH_024
no start or stop
MED_LYMPH_067
no start or stop
ABD_LYMPH_007
no start or stop
ABD_LYMPH_007
no start or stop
MED_LYMPH_055
no start or stop
MED_LYMPH_073
no start or stop
MED_LYMPH_024
no start or stop
MED_LYMPH_063
no start or stop
MED_LYMPH_067
no start or stop
MED_LYMPH_055
no start or stop
ABD_LYMPH_016
no start

In [ ]:
x,y = train_generator.__getitem__(0)

In [ ]:
plot_scroll_Image(x)

### Alright, lets make our model!

In [ ]:
from Easy_VGG16_UNet.Keras_Fine_Tune_VGG_16_Liver import VGG_16
from Visualizing_Model.Visualing_Model import visualization_model_class
from keras.optimizers import Adam
import tensorflow as tf
import keras.backend as K

In [ ]:
K.clear_session()
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
K.set_session(sess)
network = {'Layer_0': {'Encoding': [64, 64], 'Decoding': [32]},
           'Layer_1': {'Encoding': [128, 128], 'Decoding': [64]},
           'Layer_2': {'Encoding': [256, 256, 256], 'Decoding': [256]},
           'Layer_3': {'Encoding': [512, 512, 512], 'Decoding': [256]},
           'Layer_4': {'Encoding': [512, 512, 512]}}
VGG_model = VGG_16(network=network, activation='relu',filter_size=(3,3))
VGG_model.make_model()
VGG_model.load_weights()
new_model = VGG_model.created_model
model_path = os.path.join('..','Models')

## These are some tools for visualizing the model

In [ ]:
Visualizing_Class = visualization_model_class(model=new_model, save_images=True, verbose=True)

In [ ]:
Visualizing_Class.define_desired_layers(['block1_conv1','block1_conv1_activation','Output'])

In [ ]:
Visualizing_Class.predict_on_tensor(x[0,...][None,...])

In [ ]:
Visualizing_Class.plot_activations()

In [ ]:
new_model.compile(Adam(lr=5e-5),loss='categorical_crossentropy', metrics=['accuracy',dice_coef_3D])

## Freezing pre-trained layers

In [ ]:
def freeze_until_name(model,name):
    set_trainable = False
    for layer in model.layers:
        if layer.name == name:
            set_trainable = True
        layer.trainable = set_trainable
    return model
new_model = freeze_until_name(new_model,'Upsampling0_UNet')

## Checkpoint and run

A checkpoint is a way of assessing the model and determining if we should save it

In [13]:
model_name = 'VGG_16_Model'
other_aspects = [model_name,'Upsampling0_UNet_Unfrozen'] # Just a list of defining things
model_path_out = model_path_maker(model_path,other_aspects)

NameError: name 'model_path' is not defined

In [ ]:
checkpoint = ModelCheckpoint(os.path.join(model_path_out,'best-model.hdf5'), monitor='val_dice_coef_3D', verbose=1, save_best_only=True,
                              save_weights_only=False, period=5, mode='max')
# TensorboardImage lets us view the predictions of our model
tensorboard = TensorBoardImage(log_dir=model_path_out, batch_size=1, num_images=3,update_freq='epoch', 
                               data_generator=validation_generator)
callbacks = [checkpoint, tensorboard]

In [10]:
%tensorboard --logdir {"../Models"}

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 18088.

In [ ]:
#new_model.fit_generator(train_generator,epochs=1, workers=20, max_queue_size=50, validation_data=validation_generator,callbacks=callbacks)

In [ ]:
x,y = validation_generator.__getitem__(0)

In [ ]:
pred = new_model.predict(x)

In [ ]:
pred[pred<0.5] = 0
pred[pred>0] = 1

In [ ]:
plot_scroll_Image(pred[...,1])

# Now lets make our own architecture

In [ ]:
from UNet_Maker import my_UNet
from functools import partial

In [17]:
filters_start = 16
layers = 3
conv_blocks = 2
learning_rate = 1e-4

In [10]:
layers_dict = {}
filters = filters_start
pool = (4,4)
conv_block = lambda x: {'Channels': [x], 'Kernel': [(3, 3)]}
for layer in range(layers-1):
    layers_dict['Layer_{}'.format(layer)] = {'Encoding':[conv_block(filters) for _ in range(conv_blocks)],
                          'Pooling':pool,
                          'Decoding':[conv_block(filters) for _ in range(conv_blocks)]}
    filters *= 2
    pool = (2,2)
layers_dict['Base'] = {'Encoding':[conv_block(filters) for _ in range(conv_blocks)]}

In [ ]:
K.clear_session()
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
K.set_session(sess)
new_model = my_UNet(layers_dict=layers_dict,image_size=512,num_channels=train_generator.channels).created_model
new_model.compile(Adam(lr=learning_rate),loss='categorical_crossentropy', metrics=['accuracy',dice_coef_3D])

In [18]:
model_name = 'My_New_Model'
other_aspects = [model_name,'{}_Layers'.format(layers),'{}_Conv_Blocks'.format(conv_blocks),
                 '{}_Filters'.format(filters_start),'{}_Learning_Rate'.format(learning_rate)] # Just a list of defining things
model_path_out = model_path_maker(model_path,other_aspects)

In [ ]:
checkpoint = ModelCheckpoint(os.path.join(model_path_out,'best-model.hdf5'), monitor='val_dice_coef_3D', verbose=1, save_best_only=True,
                              save_weights_only=False, period=5, mode='max')
# TensorboardImage lets us view the predictions of our model
tensorboard = TensorBoardImage(log_dir=model_path_out, batch_size=1, num_images=3,update_freq='epoch', 
                               data_generator=validation_generator)
callbacks = [checkpoint, tensorboard]

In [ ]:
new_model.fit_generator(train_generator,epochs=5, workers=20, max_queue_size=50, validation_data=validation_generator,
                       callbacks=callbacks, steps_per_epoch=200)